In [ ]:
!pip install ipy_table
!pip install plotly
!pip install yellowbrick

In [ ]:
# for basic mathematics operation 
import numpy as np
import pandas as pd
from pandas import plotting

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from pandas import json_normalize
import math

# for interactive visualizations
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff
py.init_notebook_mode()
import json
from collections import Counter

# for path
import os
import copy
datafile = '../PGDnophilly_1_old.csv'

In [ ]:
h = pd.read_csv(datafile)


In [ ]:
import pandas as pd 
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt

# Header Files for Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Header Files for finding optimal linkage for clustering
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

# Header files for visualizing the clusters
from scipy.cluster.hierarchy import linkage,dendrogram,cut_tree
from yellowbrick.cluster import SilhouetteVisualizer

# Header Files for Agglomerative Clustering
from sklearn.cluster import AgglomerativeClustering

# Header Files for KMeans Custering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Header files for dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

# Header files for DBSCan
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN

sns.set(rc={'figure.figsize':(11,8)})


In [ ]:
df=pd.read_csv(datafile)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
df.head()
df=df.fillna('')
df = df.rename(columns={'u000id':'userID'})
df =df[['userID','action','types of posts','Challenges','Proficiency in DK','Proficiency in CS']]
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] 
    
challenges=[
'ASS',
'D-D',
'D-U',
'FIND',
# 'SCHE',
# 'SHARE',
'U-D',
'U-U',
'UNS', 
]

types_of_posts=[
'A-A',
'A-CONFI',
'COUR',
# 'FUA',
'Q-CLAR',
# 'Q-CONF',
#'Q-INT',
# 'Q-PUSH',
#'A-ADD',
#'A-CONFA',   
# 'OTHER',
#'Q-ADD', 
#'A-RA',    
'Q-Q',
'REPORT',
'REQUEST',  
]

renametypes={'Q-ADD':'Q-Q','Q-INT':'Q-Q','A-ADD':'A-A',
             'A-RA':'A-A','A-CONFA':'A-CONFI',
            'Q-CONF' : 'Q-Q', 'Q-PUSH' : 'Q-Q','FUA':'Q-Q'}

def renamerow(x):
    in_x=x
    for key in renametypes:
        x=x.replace(key+"@",renametypes[key]+"@")
    return x
        

df=df[['userID','Challenges','types of posts','Proficiency in DK','Proficiency in CS']]
df = df[~ (df['userID'].isin(['u00002','u00006','u00007']))]
print("total number of records",len(df))
print("No of users",len(df['userID'].unique()))
df['Challenges']=df['Challenges'].apply(lambda x : '@'.join(x.replace(",","").split())+"@")
df['types_of_posts']=df['types of posts'].apply(lambda x : '@'.join(x.replace(",","").split())+"@")
df['types_of_posts']=df['types_of_posts'].apply(lambda x : renamerow(x))



In [ ]:
df.head()

In [ ]:
df_pers=df.groupby(['userID'],as_index=False).agg( {'userID':'max'})    
for index,row in df_pers.iterrows():
    for types in list(challenges):
        types=types+"@"
        if len(df[ (df['userID'] == row['userID']) & df['Challenges'].str.contains(types)]) > 0:
            df_pers.loc[index,'Challenges_'+types]= 1
        else:
            df_pers.loc[index,'Challenges_'+types]= 0

    for types in list(types_of_posts):
        types=types+"@"
        df_pers.loc[index,'types_of_posts_'+types]=len(df[ (df['userID'] == row['userID']) & df['types_of_posts'].str.contains(types)])/len(df[ (df['userID'] == row['userID'])])
    temp=df[ (df['userID'] == row['userID']) &  ( df['Proficiency in CS'] != '' )]['Proficiency in CS'].unique()
    val = 1
    if len(temp) > 0:
        val = max(temp)
        if val == 4 or val == 5:
            val = 5
        if val == 1 or val == 2:
            val = 1
    else:
        print("no proficiency")
    df_pers.loc[index,'tProficiency in CS_'+str(int(val))]= 1

    temp=df[ (df['userID'] == row['userID']) &  ( df['Proficiency in DK'] != '' )]['Proficiency in DK'].unique()
    val = 1
    if len(temp) > 0:
        val = max(temp)
        if val == 4 or val == 5:
            val = 5
        if val == 1 or val == 2:
            val = 1
    else:
        print("no proficiency dk")
        
    df_pers.loc[index,'tProficiency in DK_'+str(int(val))]= 1
        
df_pers.head(20)
print(renametypes.keys())
print(df_pers.columns)
df_pers = df_pers.fillna(0)

In [ ]:
df_pers['tProficiency in DK_3'].unique()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
# %matplotlib notebook
df_clus = df_pers.drop(columns=['userID'])
normalized_vectors = preprocessing.normalize(df_clus)

i=3
plt.title("Principal Component Analysis of Philly User Dataset",fontsize=20)
normalized_kmeans = KMeans(n_clusters=i).fit(normalized_vectors)
print(i,'Cosine kmeans:{}'.format(silhouette_score(normalized_vectors,
                                          normalized_kmeans.labels_,
                                          metric='cosine')))
df_clus['cluster']=normalized_kmeans.labels_

targets = list(df_clus['cluster'])
# print(targets)
ranking = np.random.normal(size=len(df_clus))

X_scaled = normalized_vectors
y = targets

pca = PCA(n_components=3)
pca.fit(X_scaled) 
X_pca = pca.transform(X_scaled) 

ex_variance=np.var(X_pca,axis=0)
ex_variance_ratio = ex_variance/np.sum(ex_variance)
ex_variance_ratio



In [ ]:
Xax = X_pca[:,0]
Yax = X_pca[:,1]
Zax = X_pca[:,2]
print(len(df_clus[df_clus['cluster'] ==0]))
# print(Xax)
cdict = {0:'red',1:'green',2:'blue'}
labl = {0:'General',1:'Strategist',2:'Recruits'}
marker = {0:'*',1:'*',2:'*'}
alpha = {0:.3, 1:.3,2:.33}
center_pca = pca.transform(normalized_kmeans.cluster_centers_) 

print(center_pca)
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')

fig.patch.set_facecolor('white')
for l in np.unique(y):
    ix=np.where(np.asarray(y)==l)
    ax.scatter(Xax[ix], Yax[ix], Zax[ix], c=cdict[l], s=50,
           label=labl[l], marker=marker[l], alpha=alpha[l])
# print(X_pca[:,0])
# print("hi")
for l in range(3):
    ax.scatter(center_pca[:,0][l],center_pca[:,1][l], center_pca[:,2][l],c=l,marker='o',s=50)

    # for loop ends
ax.set_xlabel("First", fontsize=8)
ax.set_ylabel("Second ", fontsize=8)
ax.set_zlabel("Third ", fontsize=8)
ax.legend()
plt.show()